In [1]:
import os
import numpy as np
from sklearn.model_selection import train_test_split

from functions import import_imagedata, label_oh_tf, ImageProcessor
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, models, transforms
import torch.optim as optim
from torch.optim import lr_scheduler

from sklearn.metrics import f1_score, accuracy_score, confusion_matrix, ConfusionMatrixDisplay
from torchvision.models import vgg16
from torchvision.models import resnet101

import cv2

from torch.utils.data import DataLoader
import wandb
from fns4wandb import train_log, build_optimizer
from copy import deepcopy
from tqdm import tqdm

import random
from sklearn.metrics import f1_score, accuracy_score, confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

from torch.utils.data import DataLoader
import wandb
from fns4wandb import train_log, build_optimizer
from copy import deepcopy
from tqdm import tqdm

import pickle

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (2.0.4) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [2]:
device = "cuda:1" if torch.cuda.is_available() else "cpu"

In [42]:
model_vgg16 = vgg16(weights="IMAGENET1K_V1")
model_vgg16.classifier.pop(6)
model_vgg16.to(device)
#print(model_vgg16)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [6]:
def metrics(label, prediction): #TypeError: Singleton array tensor(3) cannot be considered a valid collection.
    
    label= np.array(label.cpu())

    predictions_np = prediction.cpu().detach().numpy()
    #y_pred' parameter of f1_score must be an array-like or a sparse matrix. Got 7 instead.
    predicted_classes = np.argmax(predictions_np, axis=0)
    #print('metrics Label:   ', label)
    #print('metrics prediction   ', predicted_classes)
    #avg_f1_score = f1_score(label, predictions_np, average='macro')
    acc = accuracy_score(label, predicted_classes)
    
    return acc

In [7]:
def preprocess_im(img_path):
    IP = ImageProcessor(device='cpu')
    img = cv2.imread(img_path) #
    img = IP.blank_padding(img, (224,224))
    img = IP.to_tensor(img).to(device)
    return img


In [8]:
# get len of 

file_path = r'/its/home/nn268/antvis/optics/AugmentedDS_IDSW/'
random_seed =1
img_len = len(os.listdir(file_path))


#print(ids[4])
x, y = import_imagedata(file_path)

x_train, x_test, y_train, y_tests = train_test_split(x,y, test_size=0.2, train_size=0.8,
                                 random_state=random_seed, shuffle=True)
x_train, x_val, y_train, y_val = train_test_split(x_train,y_train, test_size=0.1, train_size=0.8,
                                 random_state=random_seed, shuffle=True)


In [9]:
wandb.login()

wandb: Currently logged in as: naughticalnonsence (antvis). Use `wandb login --relogin` to force relogin


True

In [10]:
title = f'IDSW_VGG16_fine_80_112023'
save_dict = {'Run' : title,
            'Current_Epoch': 0}
                #r'/its/home/nn268/antvis/optics/
save_location = r'pickles/'#pickles

In [11]:
save_dict['Run']

'IDSW_VGG16_fine_80_112023'

In [58]:
class Squeeze(nn.Module):
    def __init__(self):
        super(Squeeze, self).__init__()
    
    def forward(self, x):
        # Do your print / debug stuff here
        x = x.squeeze(0)
        return x

In [62]:
config = dict(
    epochs= 80, #30, 
    learning_rate =1e-5,
    architecture ='CNN',
    optimizer= 'adam',
    weight_decay= 4e-5,
    ks = 3,
    scheduler=0.2,
    f_lin_lay = 7168 #1024*7 = 7168
)

def train_model(model, loss_fn, x_train, x_val, y_train, y_val, config, best_acc=0): #train_dl, val_dl, 
    wandb.watch(model, loss_fn, log='all', log_freq=10)
    
    lr = config['learning_rate'] #1e-5 #config.learning_rate
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)#build_optimizer(model, optimizer=torch.optim.Adam(model.parameters(), lr=lr))#config.optimizer, config.learning_rate, config.weight_decay)
    scheduler = lr_scheduler.ExponentialLR(optimizer, gamma=config['scheduler'], last_epoch=-1) #gamma=config.scheduler, last_epoch=-1)
    
    ####
    
    
    model.train()
    
    
    #losses= []
    #predictions = []
    t_loss_list = []
    v_loss_list = []
    t_predict_list = []
    v_predict_list = []
    t_accuracy_list = []
    v_accuracy_list = []
    t_label_list = []
    v_label_list = []
    #labels = []
    
    total_epochs = 0
    for epoch in tqdm(range(config['epochs'])): #config.epochs)):
        print('E   ', epoch)
        t_correct = 0
        v_correct = 0
    
        if epoch == 0:
            best_model = deepcopy(model)
        #train_ids = random.shuffle(train_ids)
        #print(type(train_ids))
        print('training...')
        for idx, img in enumerate(x_train): 
            
            #print('Img: ', img)

            x = preprocess_im(img)
            
            #print(x)

            train_prediction = model.forward(x)
     
            train_label = label_oh_tf(y_train[idx], device=device, num_classes=11) # use same index val to index y (labels) and turn into onehot encoded label
            #print(train_prediction, train_label)
        
        
            t_loss = loss_fn(train_prediction, train_label)

            if train_prediction.argmax() == train_label.argmax():
                t_correct+=1
            t_loss_list.append(t_loss)
            t_predict_list.append(train_prediction)
            train_acc = (t_correct / len(x_train))
            t_accuracy_list.append(train_acc)
            
            t_label_list.append(train_label)
            
            optimizer.zero_grad()
            t_loss.backward()
            
            optimizer.step()
            scheduler.step()
        print('validating...')
        for idx, img in enumerate(x_val):
            
            x =preprocess_im(img)

            val_prediction = model.forward(x)
            y = label_oh_tf(y_val[idx], device=device, num_classes=11)

            
            val_prediction = model.forward(x)

            v_loss = loss_fn(val_prediction, y)
            
            v_loss_list.append(v_loss)
            if val_prediction.argmax() == y.argmax():
                v_correct +=1
            v_loss_list.append(v_loss)
            v_predict_list.append(val_prediction)
            
            v_label_list.append(y)
            val_acc = (v_correct / len(y_val))
            v_accuracy_list.append(val_acc)
            
        total_epochs += epoch

        if val_acc > best_acc:
            best_acc = val_acc
            best_model = deepcopy(model)
            save_dict['Current_Epoch'] += config['epochs']
            save_dict['model.state_dict'] = model.state_dict()
            save_dict['training_samples'] = len(train_ids)
            save_dict['validation_samples'] = len(val_ids)
            save_dict['t_loss_list'] = t_loss_list
            save_dict['t_predict_list'] = t_predict_list  
            save_dict['t_accuracy_list'] = t_accuracy_list  #
            save_dict['v_loss_list'] = v_loss_list
            save_dict['v_predict_list'] = v_predict_list  #
            save_dict['v_accuracy_list'] = v_accuracy_list  #
            save_dict['t_labels'] = t_label_list
            save_dict['v_labels'] = v_label_list
            
            title = save_dict['Run']
            with open(f'{save_location}{title}.pkl', 'wb+') as f:
                pickle.dump(save_dict, f)
            print('improvment in metrics. model saved')
        

        if (epoch+1)%2==0:
            train_log(t_loss, v_loss, epoch)
            wandb.log({'train_accuracy_%': train_acc, 'epoch':epoch})
            wandb.log({'val_accuracy_%': val_acc, 'epoch':epoch})
    model= best_model
    #labels = zip(t_label_list, v_label_list)
    #losses = zip(t_loss_list, v_loss_list)
    #predictions = zip(t_predict_list, v_predict_list)
    return model,save_dict



def pipeline(config): 

    title = f"IDSW_on_Vgg16_fine"
    
    loss_list=[]
    #loss_fn = nn.CrossEntropyLoss()
    loss_fn = nn.MSELoss()
    with wandb.init(project=title, config=config):
        config = wandb.config
        model = nn.Sequential(
            model_vgg16,
            Squeeze(),
            nn.Linear(4096,11),
            nn.Softmax(dim=0),

        ).to(device)

        model, save_dict = train_model(model, loss_fn, x_train, x_val, y_train, y_val, config) #train_dl, val_dl

    return model, save_dict

In [ ]:
model,save_dict = pipeline(config) #7,168

  0%|          | 0/80 [00:00<?, ?it/s]

E    0
training...


In [ ]:
# confusion matrix


def plot_confusion(predictions:list, actual:list, title:str):
    predict_list = [int(t.argmax()) for t in predictions]
    actual = [int(l.argmax()) for l in actual]

    actual = np.array(actual)
    predict_list = np.array(predict_list)


    #FixedLocator locations (3), usually from a call to set_ticks, does not match the number of labels (11).
    print(f'\n     {title}')
    train_epoch_matrix = confusion_matrix(actual, predict_list, labels= [0,1,2,3,4,5,6,7,8,9,10])
    disp= ConfusionMatrixDisplay(train_epoch_matrix, display_labels=[0,1,2,3,4,5,6,7,8,9,10])
    disp.plot()
    plt.show()


In [ ]:
t_predict = save_dict['t_predict_list']
t_labels = save_dict['t_labels']

v_predict = save_dict['v_predict_list'] # WHY IS THERE NOTHING IN V OREDICT LIST!
v_labels = save_dict['v_labels']

plot_confusion(t_predict, t_labels, 'Train Confusion Matrix')
plot_confusion(v_predict, v_labels, 'Validation Confusion Matrix')

In [14]:
class PrintLayer(nn.Module):
    def __init__(self):
        super(PrintLayer, self).__init__()
    
    def forward(self, x):
        # Do your print / debug stuff here
        print(x)
        return x

In [59]:
model =nn.Sequential(
    PrintLayer(),
    model_vgg16,
    PrintLayer(),
    Squeeze(),
    PrintLayer(),
    nn.Linear(4096,11),
    PrintLayer(),
    nn.Softmax(dim=0),
    PrintLayer()

).to(device)

In [60]:

x = preprocess_im(x_train[0])


train_prediction = model.forward(x)

tensor([[[[0.0668, 0.0668, 0.0668,  ..., 0.0668, 0.0668, 0.0668],
          [0.0668, 0.0668, 0.0668,  ..., 0.0668, 0.0668, 0.0668],
          [0.0668, 0.0668, 0.0668,  ..., 0.0668, 0.0668, 0.0668],
          ...,
          [0.0668, 0.0668, 0.0668,  ..., 0.0668, 0.0668, 0.0668],
          [0.0668, 0.0668, 0.0668,  ..., 0.0668, 0.0668, 0.0668],
          [0.0668, 0.0668, 0.0668,  ..., 0.0668, 0.0668, 0.0668]],

         [[0.0668, 0.0668, 0.0668,  ..., 0.0668, 0.0668, 0.0668],
          [0.0668, 0.0668, 0.0668,  ..., 0.0668, 0.0668, 0.0668],
          [0.0668, 0.0668, 0.0668,  ..., 0.0668, 0.0668, 0.0668],
          ...,
          [0.0668, 0.0668, 0.0668,  ..., 0.0668, 0.0668, 0.0668],
          [0.0668, 0.0668, 0.0668,  ..., 0.0668, 0.0668, 0.0668],
          [0.0668, 0.0668, 0.0668,  ..., 0.0668, 0.0668, 0.0668]],

         [[0.0668, 0.0668, 0.0668,  ..., 0.0668, 0.0668, 0.0668],
          [0.0668, 0.0668, 0.0668,  ..., 0.0668, 0.0668, 0.0668],
          [0.0668, 0.0668, 0.0668,  ..., 0

In [40]:
model.state_dict()

OrderedDict([('2.weight',
              tensor([[[[-5.5373e-01,  1.4270e-01,  5.2896e-01],
                        [-5.8312e-01,  3.5655e-01,  7.6566e-01],
                        [-6.9022e-01, -4.8019e-02,  4.8409e-01]],
              
                       [[ 1.7548e-01,  9.8630e-03, -8.1413e-02],
                        [ 4.4089e-02, -7.0323e-02, -2.6035e-01],
                        [ 1.3239e-01, -1.7279e-01, -1.3226e-01]],
              
                       [[ 3.1303e-01, -1.6591e-01, -4.2752e-01],
                        [ 4.7519e-01, -8.2677e-02, -4.8700e-01],
                        [ 6.3203e-01,  1.9308e-02, -2.7753e-01]]],
              
              
                      [[[ 2.3254e-01,  1.2666e-01,  1.8605e-01],
                        [-4.2805e-01, -2.4349e-01,  2.4628e-01],
                        [-2.5066e-01,  1.4177e-01, -5.4864e-03]],
              
                       [[-1.4076e-01, -2.1903e-01,  1.5041e-01],
                        [-8.4127e-01, -3.5176e-01